In [ ]:
# !pip install torchvision

In [1]:
import torch 
from pathlib import Path
from PIL import Image
import pandas as pd
import os

In [8]:
def create_classLabel_dict(annotations_dir):
    
    # Set to store unique class labels
    class_labels = set()
    label_dict= {}

    # Iterate through annotation files
    with open(annotations_dir, 'r') as f:

        # Parse annotation file (example assumes annotations are in CSV format)
        lines = f.readlines()
        for i, line in enumerate(lines):

            # Extract class label from annotation
            class_label = line.split(',')[3]  # Assuming class label is in first column
            class_labels.add(class_label.strip())


    # Write class labels to LABELS_dict file
        for idx, label in enumerate(sorted(class_labels)):
            label_dict[label]= idx
    #         f.write(label + '\n')
    
    return label_dict

In [12]:
# Path to directory containing annotations
annotations_dir = r"D:\Datasets\Subset_Aquariam\val\annotations.csv"

annotations_df= pd.read_csv(annotations_dir)
print(annotations_df['filename'])
# Define class label mapping
class_mapping = create_classLabel_dict(annotations_dir)
# class_mapping

# Function to convert bounding box coordinates to YOLO format
def convert_to_yolo_format(bbox, image_width, image_height):
    xmin, ymin, xmax, ymax = bbox
    center_x = (xmin + xmax) / (2 * image_width)
    center_y = (ymin + ymax) / (2 * image_height)
    width = (xmax - xmin) / image_width
    height = (ymax - ymin) / image_height
    
    return center_x, center_y, width, height



0     IMG_2277_jpeg_jpg.rf.86c72d6192da48d941ffa957f...
1     IMG_2277_jpeg_jpg.rf.86c72d6192da48d941ffa957f...
2     IMG_2277_jpeg_jpg.rf.86c72d6192da48d941ffa957f...
3     IMG_2277_jpeg_jpg.rf.86c72d6192da48d941ffa957f...
4     IMG_2277_jpeg_jpg.rf.86c72d6192da48d941ffa957f...
                            ...                        
90    IMG_2325_jpeg_jpg.rf.a8ebf587a5ae7d2f8c58f9775...
91    IMG_2325_jpeg_jpg.rf.a8ebf587a5ae7d2f8c58f9775...
92    IMG_2325_jpeg_jpg.rf.a8ebf587a5ae7d2f8c58f9775...
93    IMG_2325_jpeg_jpg.rf.a8ebf587a5ae7d2f8c58f9775...
94    IMG_2325_jpeg_jpg.rf.a8ebf587a5ae7d2f8c58f9775...
Name: filename, Length: 95, dtype: object


In [13]:
# Iterate through each image annotation
for index, row in annotations_df.iterrows():
    image_filename = row['filename']
    image_width = row['width']
    image_height = row['height']
    class_label = row['class']

    # Convert class label to integer ID
    class_id = class_mapping[class_label]

    # Convert bounding box coordinates to YOLO format
    bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
    yolo_bbox = convert_to_yolo_format(bbox, image_width, image_height)

    # Write YOLO format label to text file
#     baseurl= "D:\Datasets\Yolov5_labels\"
    label_filename = r"D:\Datasets\Subset_Aquariam\val_label\\" + os.path.splitext(image_filename)[0] + '.txt'
    
    with open(label_filename, 'a') as f:
        label_line = f"{class_id} {' '.join(map(str, yolo_bbox))}\n"
        f.write(label_line)


In [ ]:
import torch  # YOLOv5 implemented using pytorch
from IPython.display import Image #this is to render predictions

In [ ]:
!git clone https://github.com/ultralytics/yolov5 

In [ ]:
import os
from random import choice
import shutil

#arrays to store file names
imgs =[]
xmls =[]

#setup dir names
trainPath = 'D:/yolov5_traffic_sign_detection/dataset/images/train'
valPath = 'D:/yolov5_traffic_sign_detection/dataset/images/val'
crsPath = 'D:/yolov5_traffic_sign_detection/data/ts/' #dir where images and annotations stored

#setup ratio (val ratio = rest of the files in origin dir after splitting into train and test)
train_ratio = 0.8
val_ratio = 0.2


#total count of imgs
totalImgCount = len(os.listdir(crsPath))/2

#soring files to corresponding arrays
for (dirname, dirs, files) in os.walk(crsPath):
    for filename in files:
        if filename.endswith('.txt'):
            xmls.append(filename)
        else:
            imgs.append(filename)


#counting range for cycles
countForTrain = int(len(imgs)*train_ratio)
countForVal = int(len(imgs)*val_ratio)
print("training images are : ",countForTrain)
print("Validation images are : ",countForVal)

In [ ]:
trainimagePath = 'D:/yolov5_traffic_sign_detection/dataset/images/train'
trainlabelPath = 'D:/yolov5_traffic_sign_detection/dataset/labels/train'
valimagePath = 'D:/yolov5_traffic_sign_detection/dataset/images/val'
vallabelPath = 'D:/yolov5_traffic_sign_detection/dataset/labels/val'
#cycle for train dir
for x in range(countForTrain):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))


    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)



In [ ]:
#cycle for test dir   
for x in range(countForVal):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    
    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)

#rest of files will be validation files, so rename origin dir to val dir
#os.rename(crsPath, valPath)
shutil.move(crsPath, valPath) 

In [ ]:
!python train.py --img 415 --batch 16 --epochs 30 --data dataset.yaml --weights yolov5s.pt --cache